In [1]:
import pandas as pd
import torch
from PIL import Image
from mmdet.models.utils import weighted_boxes_fusion

In [2]:
def pred_df_to_tensor(df, img_ids=None, vocab={"NEG": 0, "Trophozoite": 1, "WBC": 2}):
    preds = []
    if img_ids is None:
        img_ids = df["Image_ID"].drop_duplicates().sort_values()
    for img_id in img_ids:
        tmp = df[df["Image_ID"] == img_id]
        boxes = torch.tensor(tmp[["xmin", "ymin", "xmax", "ymax"]].values)
        labels = torch.Tensor([vocab[i] for i in tmp["class"]]).long()
        conf = torch.tensor(list(tmp["confidence"]))
        preds.append({"boxes": boxes, "labels": labels, "scores": conf})
    return preds

def pred_tensor_to_df(preds, img_ids, vocab=['Trophozoite', 'WBC']):
    df = []
    for i, pred in enumerate(preds):
        img_id = img_ids[i]
        tmp = pd.DataFrame(pred["boxes"].numpy().tolist(), columns=["xmin", "ymin", "xmax", "ymax"])
        tmp["Image_ID"] = img_id
        tmp["confidence"] = pred["scores"].numpy().tolist()
        tmp["class"] = list(map(lambda x: vocab[x], pred["labels"]))
        tmp = tmp[["Image_ID", "class", "confidence", "ymin", "xmin", "ymax", "xmax"]]
        df.append(tmp)
    return pd.concat(df, ignore_index=True)

def unpack_preds(preds):
    keys = list(preds[0].keys())
    out = {k: [] for k in keys}
    for p in preds:
        for k in keys:
            out[k].append(p[k])
    return out

In [3]:
df_tst = pd.read_csv("Test.csv")
tst_img_ids = list(df_tst["Image_ID"])

df_preds1 = pd.read_csv("submission_ddq_swin_1cycle_epoch18.csv")
df_preds2 = pd.read_csv("submission_dino_swin_1cycle_epoch15.csv")
df_preds3 = pd.read_csv("submission_rtdetr.csv")

In [4]:
tst_preds1 = pred_df_to_tensor(df_preds1, tst_img_ids)
tst_preds2 = pred_df_to_tensor(df_preds2, tst_img_ids)
tst_preds3 = pred_df_to_tensor(df_preds3, tst_img_ids)

In [5]:
tst_preds1_ = unpack_preds(tst_preds1)
tst_preds2_ = unpack_preds(tst_preds2)
tst_preds3_ = unpack_preds(tst_preds3)

In [6]:
fused_preds = []
for i in range(len(tst_preds1_["boxes"])):
    boxes, scores, labels = weighted_boxes_fusion(
        [tst_preds1_["boxes"][i],  tst_preds2_["boxes"][i],  tst_preds3_["boxes"][i]],
        [tst_preds1_["scores"][i], tst_preds2_["scores"][i], tst_preds3_["scores"][i]],
        [tst_preds1_["labels"][i], tst_preds2_["labels"][i], tst_preds3_["labels"][i]],
        weights=[2, 2, 1],
        iou_thr=0.7,
        skip_box_thr=0.01,
        conf_type="max",
    )
    fused_preds.append({"boxes": boxes, "scores": scores, "labels": labels})

/home/code_review/mmdetection/mmdet/models/utils/wbf.py:171: UserWarning: Zero area box skipped: tensor([0., 0., 0., 0.], dtype=torch.float64).
  warnings.warn('Zero area box skipped: {}.'.format(box_part))


In [7]:
fused_preds_df = pred_tensor_to_df(fused_preds, tst_img_ids, vocab=['NEG', 'Trophozoite', 'WBC'])
fused_preds_df

/tmp/ipykernel_627/529594971.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(df, ignore_index=True)


,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,id_5n9ov0rr22.jpg,WBC,0.855329,1571.632446,62.766926,1889.769043,304.129364
1,id_5n9ov0rr22.jpg,Trophozoite,0.779805,1798.937012,1441.980347,1903.732300,1542.584229
2,id_5n9ov0rr22.jpg,Trophozoite,0.663550,1751.285645,2780.613525,1851.730713,2880.141846
3,id_5n9ov0rr22.jpg,Trophozoite,0.649789,1113.434326,2321.721924,1214.306396,2422.088135
4,id_5n9ov0rr22.jpg,Trophozoite,0.614718,1026.196533,2422.980957,1127.874023,2516.647949
...,...,...,...,...,...,...,...
185307,id_p6bg7yes86.jpg,Trophozoite,0.010425,0.254007,1260.236084,12.152036,1312.916626
185308,id_p6bg7yes86.jpg,Trophozoite,0.010388,907.342651,1319.933716,940.527832,1364.376587
185309,id_p6bg7yes86.jpg,Trophozoite,0.010229,849.794250,627.419983,886.611938,663.140625
185310,id_p6bg7yes86.jpg,Trophozoite,0.010007,0.499952,1322.503296,25.336592,1361.650024


The fused predictions removed the `NEG` predictions. We add them back here:

In [8]:
neg_preds = df_preds1[df_preds1["class"] == "NEG"].copy()
fused_preds_df = pd.concat([fused_preds_df, neg_preds], ignore_index=True)
fused_preds_df

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,id_5n9ov0rr22.jpg,WBC,0.855329,1571.632446,62.766926,1889.769043,304.129364
1,id_5n9ov0rr22.jpg,Trophozoite,0.779805,1798.937012,1441.980347,1903.732300,1542.584229
2,id_5n9ov0rr22.jpg,Trophozoite,0.663550,1751.285645,2780.613525,1851.730713,2880.141846
3,id_5n9ov0rr22.jpg,Trophozoite,0.649789,1113.434326,2321.721924,1214.306396,2422.088135
4,id_5n9ov0rr22.jpg,Trophozoite,0.614718,1026.196533,2422.980957,1127.874023,2516.647949
...,...,...,...,...,...,...,...
185602,id_btrtdkgk4r.jpg,NEG,1.000000,0.000000,0.000000,0.000000,0.000000
185603,id_straufuobm.jpg,NEG,1.000000,0.000000,0.000000,0.000000,0.000000
185604,id_nnurq35wvp.jpg,NEG,1.000000,0.000000,0.000000,0.000000,0.000000
185605,id_hdqd25rput.jpg,NEG,1.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
_ = fused_preds_df.to_csv("submission_fused.csv", index=False)

### Post-processing trick

Examining the training images by size, we see four different image sizes. It appears as if the dataset was constructed from four different sources, and that each source was perhaps labeled slighthly differently. Notably, we see:

 - All images with size (4000, 3000) are `NEG`. This probably explains why the classifier was able to classify `NEG`s so accurately. For competition purposes, this information could have been directly exploited to determine `NEG`s, but I decided to still use a ML classifier since that the correct 'real-world' approach.
 - `WBC`s are not labeled in images of size (4160, 3120). A LB boost could be achieved by removing all `WBC` predictions for images of size (4160, 3120). I didn't exploit this in my submission but I'm curious if other competitors found and utilised this 'trick'.

In [18]:
df_trn = pd.read_csv("Train.csv")
trn_img_ids = list(df_trn["Image_ID"].drop_duplicates())
df_sz = []
for img_id in trn_img_ids:
    h, w = Image.open(f"images/{img_id}").size
    df_sz.append([img_id, h, w])
df_sz = pd.DataFrame(df_sz, columns=["Image_ID", "height", "width"])

In [19]:
df_trn = df_trn.merge(df_sz)
df_trn.groupby(["height", "width"]).agg({"Image_ID": "nunique"})

,,Image_ID
height,width,
1920,1080,949
4000,3000,688
4032,3016,770
4160,3120,340


In [22]:
df_trn.groupby(["height", "width", "class"]).agg({"Image_ID": "nunique"})

Image_ID
height width class                
1920   1080  Trophozoite       909
             WBC               897
4000   3000  NEG               688
4032   3016  Trophozoite       769
             WBC               686
4160   3120  Trophozoite       340